## Homework 1

Assuming the Fermi operator:
$$
H = \sum_i \epsilon_i c_i^\dagger c_i
$$
Prove it is resistant to dephase errors

In [ ]:
import random
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, phase_damping_error
from qiskit_experiments.library import StateTomography

# Crear modelo de ruido solo con dephasing
gamma = 0.1  # Fuerza del dephasing
noise_model = NoiseModel()
dephasing = phase_damping_error(gamma)
noise_model.add_all_qubit_quantum_error(dephasing, ['id', 'ry'])  # ruido en estas puertas

# Crear simulador ruidoso
backend = AerSimulator(noise_model=noise_model)


results = []  # Almacenar resultados

for i in range(1, 10):
    theta = np.arccos(1 - 2 * np.random.uniform(0, 1))  # Uniforme sobre la esfera
    phi = np.random.uniform(0, 2 * np.pi)

    a = np.cos(theta / 2)
    b = np.exp(1j * phi) * np.sin(theta / 2)

    epsilon = [random.uniform(-1, 1), random.uniform(-1, 1)]

    qc = QuantumCircuit(2)
    qc.ry(2 * np.arccos(a), 0)
    qc.cx(0, 1)

    qst_exp = StateTomography(qc)
    qstdata = qst_exp.run(backend, seed_simulation=1000).block_for_results()

    density_matrix = qstdata.analysis_results("state").value
    rho = DensityMatrix(density_matrix)

    reduced_A = partial_trace(rho, [0]).data
    reduced_B = partial_trace(rho, [1]).data

    rho_A11 = reduced_A[1, 1]
    rho_B11 = reduced_B[1, 1]

    H_expect = (epsilon[0] * rho_A11 + epsilon[1] * rho_B11).real
    expectation_theoretical = epsilon[0] * (np.abs(a) ** 2) + epsilon[1] * (np.abs(b) ** 2)

    quantum_state = f"{a:.4f}|00⟩ + {b:.4f}|11⟩"

    results.append({
        'Iteración': i,
        'Estado Cuántico': quantum_state,
        'ε₀': epsilon[0],
        'ε₁': epsilon[1],
        '⟨H⟩ (Teórico)': expectation_theoretical,
        '⟨H⟩ (Ruido)': H_expect,
        'Diferencia (Δ)': H_expect - expectation_theoretical
    })

# Crear DataFrame
df = pd.DataFrame(results)

# Función para colorear la diferencia
def color_diff(val):
    return 'color: red' if abs(val) > 0.5 else 'color: green'

# Estilo bonito para pandas
styled_df = (df.style
             .format({
                 'ε₀': "{:.4f}",
                 'ε₁': "{:.4f}",
                 '⟨H⟩ (Teórico)': "{:.4f}",
                 '⟨H⟩ (Ruido)': "{:.4f}",
                 'Diferencia (Δ)': "{:.4f}"
             })
             .applymap(color_diff, subset=['Diferencia (Δ)'])
             .set_caption("Resultados con ruido de dephasing")
             .set_table_styles([
                 {'selector': 'th', 'props': [('background-color', '#40466e'), ('color', 'white'), ('text-align', 'center')]},
                 {'selector': 'td', 'props': [('text-align', 'center')]}
             ])
             .set_properties(**{'text-align': 'center'})
            )

styled_df


C:\Users\jalej\AppData\Local\Temp\ipykernel_14788\841414310.py:78: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(color_diff, subset=['Diferencia (Δ)'])


,Iteración,Estado Cuántico,ε₀,ε₁,⟨H⟩ (Teórico),⟨H⟩ (Ruido),Diferencia (Δ)
0,1,0.6593|00⟩ + 0.7064+0.2574j|11⟩,-0.8719,-0.2400,-0.5147,-0.6356,-0.1209
1,2,0.0878|00⟩ + -0.9880+0.1269j|11⟩,0.2414,0.1627,0.1633,0.3988,0.2356
2,3,0.8693|00⟩ + 0.4813+0.1126j|11⟩,0.5890,-0.3166,0.3678,0.0693,-0.2985
3,4,0.6919|00⟩ + -0.4918+0.5287j|11⟩,-0.2917,0.4493,0.0946,0.0796,-0.0150
4,5,0.8545|00⟩ + -0.1377+0.5009j|11⟩,0.1795,0.4576,0.2545,0.1681,-0.0865
5,6,0.4658|00⟩ + 0.2454-0.8502j|11⟩,0.5190,0.2473,0.3063,0.5958,0.2895
6,7,0.8566|00⟩ + -0.1349+0.4980j|11⟩,0.8303,-0.1930,0.5580,0.1663,-0.3917
7,8,0.9741|00⟩ + 0.1285+0.1860j|11⟩,-0.3943,0.8800,-0.3291,0.0265,0.3557
8,9,0.7244|00⟩ + 0.6698+0.1632j|11⟩,-0.0035,-0.4566,-0.2188,-0.2203,-0.0015


## Homework 2

Assume the Hamiltonian:
$$
H = \sum_i \epsilon_i Z_i
$$

where \( Z_i \) is the Pauli-Z operator acting on site \(i\).

In [58]:
import random
import numpy as np
from tabulate import tabulate
from qiskit import QuantumCircuit
import numpy as np
from qiskit.quantum_info import DensityMatrix, partial_trace,Operator,Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, phase_damping_error
from qiskit_experiments.library import StateTomography
import pandas as pd

results = [] #To store the results

for i in range(0,10):
    # Random state
    state = random_statevector(4)
    epsilon = [random.uniform(-10, 10), random.uniform(-10, 10)]

    qc = QuantumCircuit(2)
    qc.initialize(state.data, [0, 1])
    
    epsilon = [random.uniform(-10,10),random.uniform(-10,10)]
    # Obtain the density matrix to calculate the real expected value.
    statevector = Statevector.from_instruction(qc)
    # Convert to density matrix
    rho = statevector.to_operator()
    # Create Pauli-Z operators for each qubit
    Z = Operator([[1, 0], [0, -1]])  # Pauli-Z matrix
    I = Operator([[1, 0], [0, 1]])  # Identity

    # Construct H = ε₀ Z⊗I + ε₁ I⊗Z
    H = epsilon[0] * Z.tensor(I) + epsilon[1] * I.tensor(Z)

    # Calculate expectation value ⟨H⟩ = Tr(ρH)
    expectation_theoretical = np.real((rho.data @ H.data).trace())

    # Noise model with Z noisy error
    noise_model = NoiseModel()
    p_dephasing = 0.1

    # Apply to single-qubit gates (RY)
    noise_model.add_all_qubit_quantum_error(
    phase_damping_error(p_dephasing),
    ['id', 'u3']  # All single-qubit gates
    )

    # Simulator
    backend = AerSimulator(noise_model=noise_model)

    # Tomography
    qst_exp = StateTomography(qc)
    result = qst_exp.run(backend, shots=10000).block_for_results()
    rho_noisy = DensityMatrix(result.analysis_results("state").value)

    # Calculate noisy expectation value ⟨H⟩
    H_expect = np.real((rho_noisy.data @ H.data).trace())
    # Quantum state to string
    ket_str = f"({', '.join([f'{amp:.3f}' for amp in state.data])})"
    E_abs= np.abs(H_expect - expectation_theoretical)
    E_relative = np.abs(E_abs/ expectation_theoretical) if expectation_theoretical != 0 else 0

    results.append({
        'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)': ket_str
,
        r"$\epsilon_0$": epsilon[0],
        r"$\epsilon_1$": epsilon[1],
        r"$\langle H \rangle_{theo}$": expectation_theoretical,
        r"$\langle H \rangle_{noisy}$": H_expect,
        r"$E_{abs}$": E_abs,
        r"$E_{rel}$":E_relative,
    })

# DataFrame 
df = pd.DataFrame(results)

df.columns = [
    'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)',
    r"$\epsilon_0$",
    r"$\epsilon_1$",
    r"$\langle H \rangle_{theo}$",
    r"$\langle H \rangle_{noisy}$",
    r"$E_{abs}$",
    r"$E_{rel}$"  # Relative error
]

def color_diff(val):
    abs_val = abs(val)
    color = 'red' if abs_val > 0.5 else 'green'
    return f'color: {color}'

styled_df = (df.style
             .format({
                 r"$\epsilon_0$": "{:.3f}",
                 r"$\epsilon_1$": "{:.3f}",
                 r"$\langle H \rangle_{theo}$": "{:.3f}",
                 r"$\langle H \rangle_{noisy}$": "{:.3f}",
                 r"$E_{abs}$": "{:.3f}",
                 r"$E_{rel}$": "{:.3f}"
             })
             .set_caption("Results for different quantum states and epsilon values")
             .set_table_styles([
                 {'selector': 'th', 'props': [('background-color', '#40466e'), 
                                              ('color', 'white'), 
                                              ('text-align', 'center')]},
                 {'selector': 'td', 'props': [('text-align', 'center')]},
             ])
             .set_properties(**{'text-align': 'center'})
            )

styled_df



,"Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$)",$\epsilon_0$,$\epsilon_1$,$\langle H \rangle_{theo}$,$\langle H \rangle_{noisy}$,$E_{abs}$,$E_{rel}$
0,"(0.371+0.119j, -0.316+0.117j, -0.001-0.159j, 0.455+0.709j)",-7.980,7.019,-0.785,-0.742,0.043,0.055
1,"(-0.330+0.468j, -0.629+0.208j, 0.002-0.032j, -0.459+0.149j)",6.570,4.918,1.813,1.835,0.022,0.012
2,"(-0.537+0.239j, -0.459-0.116j, 0.298-0.141j, -0.200-0.531j)",4.365,-0.533,0.657,0.675,0.018,0.027
3,"(0.089-0.659j, -0.577+0.005j, -0.271-0.170j, -0.348-0.025j)",9.567,9.363,6.127,6.029,0.098,0.016
4,"(0.019+0.135j, 0.452+0.397j, 0.681-0.232j, 0.317+0.043j)",9.332,0.826,-2.168,-2.165,0.002,0.001
5,"(0.159+0.195j, 0.504+0.476j, 0.210-0.441j, 0.304+0.353j)",-6.102,-3.459,0.834,0.844,0.011,0.013
6,"(-0.298+0.297j, 0.595-0.210j, 0.143-0.182j, -0.567-0.223j)",-4.609,4.369,-3.051,-3.076,0.025,0.008
7,"(0.079+0.163j, 0.104+0.457j, 0.576-0.197j, 0.367-0.492j)",-2.722,8.787,-0.341,-0.343,0.003,0.007
8,"(0.399-0.847j, 0.128-0.148j, -0.055-0.021j, 0.145+0.246j)",8.157,9.525,14.015,14.010,0.004,0.000
9,"(0.226+0.362j, -0.387-0.009j, -0.334+0.036j, -0.333-0.666j)",-1.333,2.139,-0.429,-0.428,0.000,0.001


In [59]:
# Create pandas table
df = pd.DataFrame(results)

# Generate LaTeX table ready to copy and paste
latex_table = df.to_latex(index=False, escape=False, column_format='c' * len(df.columns), float_format="%.3f")
# Optional: add table environment and caption
latex_code = r"""
\begin{table}[H]
    \centering
    \caption{Comparison of noiseless and with phase damping noise expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_Z}
""" + latex_table + r"""
\end{table}
"""

# Print to copy and paste
print(latex_code)


\begin{table}[H]
    \centering
    \caption{Comparison of noiseless and with phase damping noise expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_Z}
\begin{tabular}{ccccccc}
\toprule
Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$) & $\epsilon_0$ & $\epsilon_1$ & $\langle H \rangle_{theo}$ & $\langle H \rangle_{noisy}$ & $E_{abs}$ & $E_{rel}$ \\
\midrule
(0.371+0.119j, -0.316+0.117j, -0.001-0.159j, 0.455+0.709j) & -7.980 & 7.019 & -0.785 & -0.742 & 0.043 & 0.055 \\
(-0.330+0.468j, -0.629+0.208j, 0.002-0.032j, -0.459+0.149j) & 6.570 & 4.918 & 1.813 & 1.835 & 0.022 & 0.012 \\
(-0.537+0.239j, -0.459-0.116j, 0.298-0.141j, -0.200-0.531j) & 4.365 & -0.533 & 0.657 & 0.675 & 0.018 & 0.027 \\
(0.089-0.659j, -0.577+0.005j, -0.271-0.170j, -0.348-0.025j) & 9.567 & 9.363 & 6.127 & 6.029 & 0.098 & 0.016 \\
(0.019+0.135j, 0.452+0.397j, 0.681-0.232j, 0.317+0.043j) & 9.332 & 0.826 & -2.168 & -2.165 & 0.002 & 0.001 \\
(0.15

## Homework 3

Assume the Hamiltonian:
$$
H = \sum_i \epsilon_i X_i
$$

where \( X_i \) is the Pauli-X operator acting on site \(i\).

Prove this is resistant for X dephasing

In [60]:
import numpy as np
import random
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, random_statevector, DensityMatrix
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
from qiskit_experiments.library import StateTomography

results = []

for i in range(10):  # Número de iteraciones

    # Random state
    state = random_statevector(4)
    epsilon = [round(random.uniform(-10, 10), 3), round(random.uniform(-10, 10), 3)]  # 3 decimal places

    qc = QuantumCircuit(2)
    qc.initialize(state.data, [0, 1])

    # Convertir a matriz densidad
    rho = DensityMatrix(state)

    # Define H = ε₀ X⊗I + ε₁ I⊗X
    X = Operator([[0, 1], [1, 0]])
    I = Operator([[1, 0], [0, 1]])
    H = epsilon[0] * X.tensor(I) + epsilon[1] * I.tensor(X)

    # Theoretical expected value
    expectation_theoretical = np.real((rho.data @ H.data).trace())

    # Noisy mode bit flip error
    noise_model = NoiseModel()
    p_error = 0.1
    x_error = pauli_error([('X', p_error), ('I', 1 - p_error)])
    noise_model.add_all_qubit_quantum_error(x_error, ['id', 'u3','x'])

    backend = AerSimulator(noise_model=noise_model)

    # Quantum tomography
    qst_exp = StateTomography(qc)
    result = qst_exp.run(backend, shots=50000).block_for_results()
    rho_noisy = DensityMatrix(result.analysis_results("state").value)

    # Noisy expected value
    H_expect = np.real((rho_noisy.data @ H.data).trace())

    # Quantum state to string
    ket_str = f"({', '.join([f'{amp:.3f}' for amp in state.data])})"
    E_abs= np.abs(H_expect - expectation_theoretical)
    E_relative = np.abs(E_abs/ expectation_theoretical) if expectation_theoretical != 0 else 0

    results.append({
        'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)': ket_str
,
        r"$\epsilon_0$": epsilon[0],
        r"$\epsilon_1$": epsilon[1],
        r"$\langle H \rangle_{theo}$": expectation_theoretical,
        r"$\langle H \rangle_{noisy}$": H_expect,
        r"$E_{abs}$": E_abs,
        r"$E_{rel}$":E_relative,
    })

# DataFrame 
df = pd.DataFrame(results)

df.columns = [
    'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)',
    r"$\epsilon_0$",
    r"$\epsilon_1$",
    r"$\langle H \rangle_{theo}$",
    r"$\langle H \rangle_{noisy}$",
    r"$E_{abs}$",
    r"$E_{rel}$"  # Relative error
]


styled_df = (df.style
             .format({
                 r"$\epsilon_0$": "{:.3f}",
                 r"$\epsilon_1$": "{:.3f}",
                 r"$\langle H \rangle_{theo}$": "{:.3f}",
                 r"$\langle H \rangle_{noisy}$": "{:.3f}",
                 r"$E_{abs}$": "{:.3f}",
                 r"$E_{rel}$": "{:.3f}"
             })
             .set_caption("Results for different quantum states and epsilon values")
             .set_table_styles([
                 {'selector': 'th', 'props': [('background-color', '#40466e'), 
                                              ('color', 'white'), 
                                              ('text-align', 'center')]},
                 {'selector': 'td', 'props': [('text-align', 'center')]},
             ])
             .set_properties(**{'text-align': 'center'})
            )

styled_df



,"Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$)",$\epsilon_0$,$\epsilon_1$,$\langle H \rangle_{theo}$,$\langle H \rangle_{noisy}$,$E_{abs}$,$E_{rel}$
0,"(0.147-0.255j, -0.176-0.519j, 0.145+0.370j, 0.601+0.309j)",-2.594,1.300,2.556,2.054,0.502,0.197
1,"(-0.241+0.327j, -0.190+0.604j, -0.280-0.049j, 0.249+0.540j)",-6.956,1.452,-4.170,-3.324,0.846,0.203
2,"(0.393+0.182j, -0.271+0.407j, 0.280+0.416j, 0.564+0.065j)",1.943,-4.822,-1.238,-1.014,0.224,0.181
3,"(-0.041+0.645j, 0.056-0.272j, -0.439-0.256j, 0.282-0.410j)",-2.468,9.261,-3.539,-2.883,0.657,0.186
4,"(0.228+0.701j, -0.093-0.066j, -0.314+0.225j, -0.217+0.497j)",4.572,0.006,0.672,0.543,0.129,0.191
5,"(0.059+0.220j, 0.409+0.581j, 0.292-0.360j, -0.147-0.455j)",1.765,5.466,1.620,1.303,0.317,0.196
6,"(-0.717+0.152j, 0.156-0.236j, -0.217-0.461j, 0.350-0.020j)",0.909,7.238,-2.836,-2.264,0.571,0.201
7,"(0.390-0.103j, -0.034-0.363j, 0.560+0.300j, 0.285-0.469j)",2.898,-1.299,1.906,1.520,0.386,0.202
8,"(0.126+0.585j, -0.062-0.109j, -0.715+0.215j, -0.139+0.221j)",-3.276,-0.681,-0.233,-0.175,0.058,0.251
9,"(0.531+0.019j, 0.234-0.580j, -0.414-0.265j, -0.106-0.272j)",-0.067,6.755,3.108,2.502,0.606,0.195


In [56]:
# Create pandas table
df = pd.DataFrame(results)

# Generate LaTeX table ready to copy and paste
latex_table = df.to_latex(index=False, escape=False, column_format='c' * len(df.columns), float_format="%.3f")
# Optional: add table environment and caption
latex_code = r"""
\begin{table}[H]
    \centering
    \caption{Numerical comparison of noiseless and bit-flip-affected expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_x}
""" + latex_table + r"""
\end{table}
"""

# Print to copy and paste
print(latex_code)


\begin{table}[H]
    \centering
    \caption{Numerical comparison of noiseless and bit-flip-affected expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_x}
\begin{tabular}{ccccccc}
\toprule
Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$) & $\epsilon_0$ & $\epsilon_1$ & $\langle H \rangle_{theo}$ & $\langle H \rangle_{noisy}$ & $E_{abs}$ & $E_{rel}$ \\
\midrule
(0.920+0.030j, 0.237-0.274j, -0.050-0.035j, 0.060-0.118j) & -3.510 & 2.880 & 1.219 & 0.979 & 0.240 & 0.197 \\
(-0.206+0.193j, 0.785+0.456j, -0.196-0.133j, 0.194-0.043j) & 5.264 & 9.170 & -0.394 & -0.321 & 0.073 & 0.185 \\
(-0.242+0.577j, 0.484-0.179j, -0.231-0.086j, 0.249-0.468j) & 7.376 & -6.902 & 6.396 & 5.114 & 1.282 & 0.200 \\
(-0.191+0.450j, -0.246+0.336j, 0.085-0.010j, -0.713+0.269j) & -9.995 & -0.494 & -5.039 & -4.003 & 1.035 & 0.206 \\
(0.167+0.227j, -0.041-0.273j, -0.241+0.844j, 0.153+0.226j) & -0.209 & 7.413 & 1.227 & 0.997 & 0.230 & 0.188 \\


## Homework 4

Search a Hamiltonian that is resistant for a general dephasing


I suppose that the resistant H is:

$$
H = \epsilon \left( X_1 X_2 + Y_1 Y_2 + Z_1 Z_2 \right)
$$


In [63]:
import random
import pandas as pd

from qiskit_aer.noise import NoiseModel, amplitude_damping_error  # Changed to X error
from qiskit import QuantumCircuit
import numpy as np
from qiskit.quantum_info import DensityMatrix, partial_trace, Operator, Statevector,Pauli
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, amplitude_damping_error  # Changed to X error
from qiskit_experiments.library import StateTomography
from qiskit.visualization import plot_bloch_multivector,plot_state_qsphere
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit_ibm_runtime.fake_provider import FakePerth

from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography

results = [] #To store the results

for i in range(0,30):
    # Random state
    state = random_statevector(4)
    epsilon = [round(random.uniform(-10, 10), 3), round(random.uniform(-10, 10), 3)]  # 3 decimal places
    # Circuit (prepare state in X basis for better effect visualization)
    qc = QuantumCircuit(2)
    qc.initialize(state.data, [0, 1])

    # Convert to density matrix
    rho = DensityMatrix(state)


    H = Operator(Pauli('XX')) + Operator(Pauli('YY')) + Operator(Pauli('ZZ'))

    # Calculate expectation value ⟨H⟩ = Tr(ρH)
    expectation_theoretical = np.real((rho.data @ H.data).trace())

    # Now calculate the expected value using the simulator with random noise


    backend = AerSimulator.from_backend(FakePerth()) 


    qst_exp = StateTomography(qc) #We are going to use quantum state tomography to obatin the density matrix
    qstdata= qst_exp.run(backend,seed_simulation= 300000).block_for_results()


    state_result = qstdata.analysis_results("state")
    density_matrix= state_result.value

    rho_noisy= DensityMatrix(density_matrix)

    # Calculate noisy expectation value ⟨H⟩
    H_expect = np.real((rho_noisy.data @ H.data).trace())
 

    ket_str = f"({', '.join([f'{amp:.3f}' for amp in state.data])})"
    E_abs= np.abs(H_expect - expectation_theoretical)
    E_relative = np.abs(E_abs/ expectation_theoretical) if expectation_theoretical != 0 else 0

    results.append({
        'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)': ket_str
,
        r"$\epsilon_0$": epsilon[0],
        r"$\epsilon_1$": epsilon[1],
        r"$\langle H \rangle_{theo}$": expectation_theoretical,
        r"$\langle H \rangle_{noisy}$": H_expect,
        r"$E_{abs}$": E_abs,
        r"$E_{rel}$":E_relative,
    })

# DataFrame 
df = pd.DataFrame(results)

df.columns = [
    'Quantum State($\\{|00\\rangle, |01\\rangle, |10\\rangle, |11\\rangle\\}$)',
    r"$\epsilon_0$",
    r"$\epsilon_1$",
    r"$\langle H \rangle_{theo}$",
    r"$\langle H \rangle_{noisy}$",
    r"$E_{abs}$",
    r"$E_{rel}$"  # Relative error
]

def color_diff(val):
    abs_val = abs(val)
    color = 'red' if abs_val > 0.5 else 'green'
    return f'color: {color}'

styled_df = (df.style
             .format({
                 r"$\epsilon_0$": "{:.3f}",
                 r"$\epsilon_1$": "{:.3f}",
                 r"$\langle H \rangle_{theo}$": "{:.3f}",
                 r"$\langle H \rangle_{noisy}$": "{:.3f}",
                 r"$E_{abs}$": "{:.3f}",
                 r"$E_{rel}$": "{:.3f}"
             })
             .set_caption("Results for different quantum states and epsilon values")
             .set_table_styles([
                 {'selector': 'th', 'props': [('background-color', '#40466e'), 
                                              ('color', 'white'), 
                                              ('text-align', 'center')]},
                 {'selector': 'td', 'props': [('text-align', 'center')]},
             ])
             .set_properties(**{'text-align': 'center'})
            )

styled_df

,"Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$)",$\epsilon_0$,$\epsilon_1$,$\langle H \rangle_{theo}$,$\langle H \rangle_{noisy}$,$E_{abs}$,$E_{rel}$
0,"(-0.175+0.656j, 0.352-0.025j, -0.048+0.235j, 0.187+0.567j)",2.457,1.478,0.546,0.422,0.124,0.227
1,"(0.021-0.297j, 0.570+0.026j, -0.561-0.266j, 0.385+0.228j)",3.030,7.735,-1.731,-1.437,0.294,0.170
2,"(-0.626-0.126j, -0.177+0.566j, 0.345-0.013j, 0.337+0.092j)",-8.150,-5.518,-0.214,-0.166,0.048,0.223
3,"(-0.085-0.036j, 0.350-0.532j, -0.267-0.336j, 0.397+0.495j)",-0.735,6.634,0.164,0.238,0.074,0.450
4,"(0.136-0.246j, -0.221-0.407j, 0.763+0.211j, -0.040+0.282j)",-2.450,-7.123,-1.697,-1.484,0.213,0.125
5,"(0.562+0.021j, 0.404+0.048j, -0.461+0.345j, 0.242-0.358j)",1.569,5.668,-0.674,-0.578,0.096,0.142
6,"(0.027-0.410j, -0.817-0.123j, 0.192+0.244j, 0.219+0.066j)",-0.657,-1.020,-1.307,-1.058,0.249,0.190
7,"(0.589-0.077j, 0.142+0.651j, 0.182+0.230j, 0.334-0.073j)",2.297,-5.112,0.643,0.588,0.055,0.086
8,"(0.262+0.305j, -0.117+0.238j, 0.691+0.481j, -0.041-0.237j)",-3.592,-4.201,-0.426,-0.436,0.010,0.024
9,"(0.603+0.388j, -0.143+0.231j, 0.086+0.037j, -0.237+0.590j)",-6.846,-0.776,0.820,0.772,0.048,0.058


In [64]:
# Create pandas table
df = pd.DataFrame(results)

# Generate LaTeX table ready to copy and paste
latex_table = df.to_latex(index=False, escape=False, column_format='c' * len(df.columns), float_format="%.3f")
# Optional: add table environment and caption
latex_code = r"""
\begin{table}[H]
    \centering
    \caption{Numerical comparison of noiseless and general error affected expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_general}
""" + latex_table + r"""
\end{table}
"""

# Print to copy and paste
print(latex_code)


\begin{table}[H]
    \centering
    \caption{Numerical comparison of noiseless and general error affected expectation values computed for random state preparations and $\epsilon$ values.}
    \label{tab:qst_results_general}
\begin{tabular}{ccccccc}
\toprule
Quantum State($\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$) & $\epsilon_0$ & $\epsilon_1$ & $\langle H \rangle_{theo}$ & $\langle H \rangle_{noisy}$ & $E_{abs}$ & $E_{rel}$ \\
\midrule
(-0.175+0.656j, 0.352-0.025j, -0.048+0.235j, 0.187+0.567j) & 2.457 & 1.478 & 0.546 & 0.422 & 0.124 & 0.227 \\
(0.021-0.297j, 0.570+0.026j, -0.561-0.266j, 0.385+0.228j) & 3.030 & 7.735 & -1.731 & -1.437 & 0.294 & 0.170 \\
(-0.626-0.126j, -0.177+0.566j, 0.345-0.013j, 0.337+0.092j) & -8.150 & -5.518 & -0.214 & -0.166 & 0.048 & 0.223 \\
(-0.085-0.036j, 0.350-0.532j, -0.267-0.336j, 0.397+0.495j) & -0.735 & 6.634 & 0.164 & 0.238 & 0.074 & 0.450 \\
(0.136-0.246j, -0.221-0.407j, 0.763+0.211j, -0.040+0.282j) & -2.450 & -7.123 & -1.697 & -1.484 & 0.213

In [13]:
import qiskit
print(qiskit.__version__)

1.4.2
